# Boston Housing Dataset Regression Analysis

## ⚠️ Important Note
**The Boston Housing dataset has been deprecated in scikit-learn due to ethical concerns regarding its data collection methodology and potential for reinforcing harmful biases. This notebook is provided for educational purposes to understand regression techniques, but should not be used as a reference for real-world housing price prediction without addressing these ethical considerations.**

## Overview
The Boston Housing dataset contains information about housing in the area of Boston Mass. It was originally published in 1978 and contains 506 samples with 13 features each. The task is to predict the median value of owner-occupied homes (in thousands of dollars).

## Dataset Details
- **Samples**: 506 housing records
- **Features**: 13 numerical features
- **Target**: Median home value (continuous variable)
- **Task**: Regression (predicting continuous values)
- **Application**: Real estate valuation, economic analysis

## Features Description
1. **CRIM**: Per capita crime rate by town
2. **ZN**: Proportion of residential land zoned for lots over 25,000 sq.ft.
3. **INDUS**: Proportion of non-retail business acres per town
4. **CHAS**: Charles River dummy variable (1 if tract bounds river; 0 otherwise)
5. **NOX**: Nitric oxides concentration (parts per 10 million)
6. **RM**: Average number of rooms per dwelling
7. **AGE**: Proportion of owner-occupied units built prior to 1940
8. **DIS**: Weighted distances to five Boston employment centres
9. **RAD**: Index of accessibility to radial highways
10. **TAX**: Full-value property-tax rate per $10,000
11. **PTRATIO**: Pupil-teacher ratio by town
12. **B**: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13. **LSTAT**: % lower status of the population

## Step 1: Import Required Libraries
Import libraries for regression analysis, visualization, and model evaluation.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.model_selection import (
    train_test_split, cross_val_score, GridSearchCV, 
    learning_curve, validation_curve
)
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet
)
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    explained_variance_score
)
from sklearn.decomposition import PCA
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("⚠️  Note: This dataset is deprecated due to ethical concerns.")
print("This analysis is for educational purposes only.")

## Step 2: Load and Explore the Dataset
Load the Boston Housing dataset and perform initial exploration.

In [ ]:
# Load the Boston Housing dataset with data_home parameter to handle deprecation
try:
    boston = load_boston()
    print("Dataset loaded successfully.")
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("The Boston Housing dataset has been removed from scikit-learn.")
    print("Creating synthetic data for demonstration purposes...")
    
    # Create synthetic data that mimics the Boston Housing dataset structure
    np.random.seed(42)
    n_samples = 506
    
    # Create synthetic features
    data = np.column_stack([
        np.random.exponential(3, n_samples),  # CRIM
        np.random.exponential(12, n_samples),  # ZN
        np.random.beta(2, 5, n_samples) * 25,  # INDUS
        np.random.binomial(1, 0.07, n_samples),  # CHAS
        np.random.normal(0.55, 0.1, n_samples),  # NOX
        np.random.normal(6.3, 0.7, n_samples),  # RM
        np.random.beta(5, 2, n_samples) * 100,  # AGE
        np.random.exponential(4, n_samples),  # DIS
        np.random.choice(range(1, 25), n_samples),  # RAD
        np.random.normal(400, 150, n_samples),  # TAX
        np.random.normal(18, 2, n_samples),  # PTRATIO
        np.random.beta(10, 1, n_samples) * 400,  # B
        np.random.beta(2, 8, n_samples) * 35  # LSTAT
    ])
    
    # Create synthetic target (house prices) based on features
    target = (50 - data[:, 0] * 0.1 + data[:, 5] * 8 - data[:, 12] * 0.5 + 
              np.random.normal(0, 5, n_samples))
    target = np.clip(target, 5, 50)  # Clip to reasonable range
    
    # Create feature names
    feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 
                    'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
    
    # Create a boston-like object
    class SyntheticBoston:
        def __init__(self, data, target, feature_names):
            self.data = data
            self.target = target
            self.feature_names = feature_names
    
    boston = SyntheticBoston(data, target, feature_names)
    print("Synthetic dataset created for demonstration.")

# Create a DataFrame for easier manipulation
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['target'] = boston.target
df['price'] = boston.target  # More intuitive name

print("\nDataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nTarget Variable Statistics:")
print(f"Mean house price: ${df['price'].mean():.2f}k")
print(f"Median house price: ${df['price'].median():.2f}k")
print(f"Price range: ${df['price'].min():.2f}k - ${df['price'].max():.2f}k")
print(f"Standard deviation: ${df['price'].std():.2f}k")

## Step 3: Comprehensive Data Analysis
Perform detailed statistical analysis and identify patterns in the housing data.

In [ ]:
# Statistical summary
print("Statistical Summary:")
print(df.describe())

# Check for missing values
print("\nMissing Values:")
missing_values = df.isnull().sum()
if missing_values.sum() == 0:
    print("No missing values found.")
else:
    print(missing_values[missing_values > 0])

# Outlier detection using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

print("\nOutlier Analysis:")
outlier_summary = []
for column in boston.feature_names + ['price']:
    n_outliers, lower, upper = detect_outliers_iqr(df, column)
    outlier_summary.append({
        'Feature': column,
        'Outliers': n_outliers,
        'Percentage': f"{n_outliers/len(df)*100:.1f}%",
        'Lower_Bound': f"{lower:.2f}",
        'Upper_Bound': f"{upper:.2f}"
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df.to_string(index=False))

# Feature scaling analysis
print("\nFeature Scaling Analysis:")
scaling_analysis = pd.DataFrame({
    'Feature': boston.feature_names,
    'Mean': df[boston.feature_names].mean(),
    'Std': df[boston.feature_names].std(),
    'Min': df[boston.feature_names].min(),
    'Max': df[boston.feature_names].max(),
    'Range': df[boston.feature_names].max() - df[boston.feature_names].min()
})

print("Features with largest ranges (indicating need for scaling):")
print(scaling_analysis.nlargest(5, 'Range')[['Feature', 'Mean', 'Std', 'Range']])

## Step 4: Comprehensive Data Visualization
Create detailed visualizations to understand relationships and distributions.

In [ ]:
# Target variable distribution analysis
plt.figure(figsize=(20, 12))

# Price distribution
plt.subplot(2, 4, 1)
plt.hist(df['price'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(df['price'].mean(), color='red', linestyle='--', label=f'Mean: ${df["price"].mean():.1f}k')
plt.axvline(df['price'].median(), color='green', linestyle='--', label=f'Median: ${df["price"].median():.1f}k')
plt.xlabel('House Price ($k)')
plt.ylabel('Frequency')
plt.title('Distribution of House Prices')
plt.legend()
plt.grid(True, alpha=0.3)

# Q-Q plot for normality check
plt.subplot(2, 4, 2)
stats.probplot(df['price'], dist="norm", plot=plt)
plt.title('Q-Q Plot: Price vs Normal Distribution')
plt.grid(True, alpha=0.3)

# Log-transformed price distribution
plt.subplot(2, 4, 3)
log_price = np.log(df['price'])
plt.hist(log_price, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
plt.xlabel('Log(House Price)')
plt.ylabel('Frequency')
plt.title('Log-Transformed Price Distribution')
plt.grid(True, alpha=0.3)

# Box plot of prices
plt.subplot(2, 4, 4)
plt.boxplot(df['price'], vert=True)
plt.ylabel('House Price ($k)')
plt.title('Price Distribution (Box Plot)')
plt.grid(True, alpha=0.3)

# Feature distributions (first 4 features)
for i, feature in enumerate(boston.feature_names[:4]):
    plt.subplot(2, 4, 5 + i)
    plt.hist(df[feature], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.title(f'Distribution of {feature}')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
plt.figure(figsize=(16, 12))

# Correlation matrix
plt.subplot(2, 2, 1)
correlation_matrix = df[boston.feature_names + ['price']].corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Matrix')

# Correlation with target variable
plt.subplot(2, 2, 2)
target_corr = correlation_matrix['price'].drop('price').sort_values(key=abs, ascending=False)
colors = ['red' if x < 0 else 'blue' for x in target_corr.values]
bars = plt.barh(range(len(target_corr)), target_corr.values, color=colors, alpha=0.7)
plt.yticks(range(len(target_corr)), target_corr.index)
plt.xlabel('Correlation with House Price')
plt.title('Feature Correlation with Target Variable')
plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)
plt.grid(True, alpha=0.3)

# Add correlation values on bars
for i, (bar, corr) in enumerate(zip(bars, target_corr.values)):
    plt.text(corr + (0.02 if corr > 0 else -0.02), i, f'{corr:.3f}', 
             va='center', ha='left' if corr > 0 else 'right', fontsize=9)

# Scatter plots of top correlated features
top_features = target_corr.head(3).index
for i, feature in enumerate(top_features):
    plt.subplot(2, 2, 3 + i)
    plt.scatter(df[feature], df['price'], alpha=0.6, s=30)
    
    # Add trend line
    z = np.polyfit(df[feature], df['price'], 1)
    p = np.poly1d(z)
    plt.plot(df[feature], p(df[feature]), "r--", alpha=0.8)
    
    plt.xlabel(feature)
    plt.ylabel('House Price ($k)')
    plt.title(f'{feature} vs Price (r={target_corr[feature]:.3f})')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print correlation insights
print("Correlation Analysis Insights:")
print(f"Strongest positive correlation: {target_corr.idxmax()} ({target_corr.max():.3f})")
print(f"Strongest negative correlation: {target_corr.idxmin()} ({target_corr.min():.3f})")

strong_corr_features = target_corr[abs(target_corr) > 0.5]
print(f"\nFeatures with strong correlation (|r| > 0.5):")
for feature, corr in strong_corr_features.items():
    print(f"  {feature}: {corr:.3f}")

In [ ]:
# Feature relationships and multicollinearity analysis
plt.figure(figsize=(20, 12))

# Pairwise relationships for selected features
important_features = ['LSTAT', 'RM', 'PTRATIO', 'INDUS', 'price']
subset_df = df[important_features]

# Create pairplot
sns.pairplot(subset_df, diag_kind='hist', plot_kws={'alpha': 0.6, 's': 30})
plt.suptitle('Pairwise Relationships - Key Features', y=1.02, fontsize=16)
plt.show()

# Analyze multicollinearity
high_corr_pairs = []
feature_corr = correlation_matrix.drop('price', axis=0).drop('price', axis=1)
for i in range(len(feature_corr.columns)):
    for j in range(i+1, len(feature_corr.columns)):
        corr_val = feature_corr.iloc[i, j]
        if abs(corr_val) > 0.7:
            high_corr_pairs.append((feature_corr.columns[i], feature_corr.columns[j], corr_val))

print("\nMulticollinearity Analysis:")
print("Feature pairs with high correlation (|r| > 0.7):")
if high_corr_pairs:
    for feat1, feat2, corr in high_corr_pairs:
        print(f"  {feat1} - {feat2}: {corr:.3f}")
else:
    print("  No feature pairs with correlation > 0.7")

# Feature importance using variance
feature_variance = df[boston.feature_names].var().sort_values(ascending=False)
print(f"\nFeatures by variance (indication of information content):")
for feature, variance in feature_variance.head(8).items():
    print(f"  {feature}: {variance:.2f}")

## Step 5: Feature Engineering and Preprocessing
Prepare features for regression modeling with scaling, selection, and transformation.

In [ ]:
# Separate features and target
X = boston.data
y = boston.target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Feature dimensions: {X_train.shape[1]}")

print(f"\nTarget variable statistics:")
print(f"Training set - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"Test set - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nFeature scaling completed:")
print(f"Original range: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"Scaled range: [{X_train_scaled.min():.2f}, {X_train_scaled.max():.2f}]")

# Feature selection using different methods
# 1. Univariate feature selection
selector_f = SelectKBest(f_regression, k=8)
X_train_selected = selector_f.fit_transform(X_train_scaled, y_train)
X_test_selected = selector_f.transform(X_test_scaled)

selected_features = np.array(boston.feature_names)[selector_f.get_support()]
feature_scores = selector_f.scores_[selector_f.get_support()]

print(f"\nTop 8 features by F-test:")
for feature, score in zip(selected_features, feature_scores):
    print(f"  {feature}: {score:.2f}")

# 2. Recursive feature elimination
from sklearn.linear_model import LinearRegression
lr_for_rfe = LinearRegression()
rfe_selector = RFE(lr_for_rfe, n_features_to_select=8)
X_train_rfe = rfe_selector.fit_transform(X_train_scaled, y_train)
X_test_rfe = rfe_selector.transform(X_test_scaled)

rfe_features = np.array(boston.feature_names)[rfe_selector.get_support()]
print(f"\nTop 8 features by RFE:")
for i, feature in enumerate(rfe_features):
    print(f"  {i+1}. {feature}")

# 3. Polynomial features (for interaction effects)
poly_features = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train_selected)  # Use selected features to avoid explosion
X_test_poly = poly_features.transform(X_test_selected)

print(f"\nPolynomial features:")
print(f"Original features: {X_train_selected.shape[1]}")
print(f"With interactions: {X_train_poly.shape[1]}")

# Compare feature selection methods
common_features = set(selected_features) & set(rfe_features)
print(f"\nCommon features between F-test and RFE: {len(common_features)}")
for feature in common_features:
    print(f"  - {feature}")

# Target transformation analysis
# Check if log transformation improves normality
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(y_train, bins=20, alpha=0.7, color='blue', edgecolor='black')
plt.title('Original Target Distribution')
plt.xlabel('House Price ($k)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
log_y_train = np.log(y_train)
plt.hist(log_y_train, bins=20, alpha=0.7, color='green', edgecolor='black')
plt.title('Log-Transformed Target')
plt.xlabel('Log(House Price)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
sqrt_y_train = np.sqrt(y_train)
plt.hist(sqrt_y_train, bins=20, alpha=0.7, color='red', edgecolor='black')
plt.title('Square Root Transformed Target')
plt.xlabel('√(House Price)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Calculate normality statistics
from scipy.stats import shapiro, normaltest

def test_normality(data, name):
    shapiro_stat, shapiro_p = shapiro(data)
    dagostino_stat, dagostino_p = normaltest(data)
    return {
        'transformation': name,
        'shapiro_p': shapiro_p,
        'dagostino_p': dagostino_p,
        'skewness': stats.skew(data),
        'kurtosis': stats.kurtosis(data)
    }

normality_tests = [
    test_normality(y_train, 'Original'),
    test_normality(log_y_train, 'Log'),
    test_normality(sqrt_y_train, 'Square Root')
]

normality_df = pd.DataFrame(normality_tests)
print("\nNormality Test Results:")
print(normality_df.round(4))
print("\nNote: Higher p-values indicate more normal distribution")
print("Skewness close to 0 and kurtosis close to 0 indicate normality")

## Step 6: Comprehensive Regression Model Training
Train multiple regression models and compare their performance.

In [ ]:
# Initialize regression models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Extra Trees': ExtraTreesRegressor(n_estimators=100, random_state=42),
    'SVR (RBF)': SVR(kernel='rbf'),
    'SVR (Linear)': SVR(kernel='linear'),
    'K-Nearest Neighbors': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=10),
    'Neural Network': MLPRegressor(random_state=42, max_iter=1000, hidden_layer_sizes=(100, 50))
}

# Different data configurations
data_configs = {
    'Original': (X_train, X_test, y_train, y_test),
    'Scaled': (X_train_scaled, X_test_scaled, y_train, y_test),
    'Selected (F-test)': (X_train_selected, X_test_selected, y_train, y_test),
    'Selected (RFE)': (X_train_rfe, X_test_rfe, y_train, y_test),
    'Polynomial': (X_train_poly, X_test_poly, y_train, y_test),
    'Log Target': (X_train_scaled, X_test_scaled, log_y_train, np.log(y_test))
}

# Regression metrics calculation
def calculate_regression_metrics(y_true, y_pred):
    return {
        'mse': mean_squared_error(y_true, y_pred),
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred),
        'explained_variance': explained_variance_score(y_true, y_pred)
    }

# Train and evaluate all models
results = {}
best_model_info = {'r2': -np.inf, 'config': None, 'model': None, 'name': None}

print("Training and evaluating regression models...\n")

for config_name, (X_tr, X_te, y_tr, y_te) in data_configs.items():
    print(f"{'='*60}")
    print(f"Configuration: {config_name}")
    print(f"{'='*60}")
    
    config_results = {}
    
    for model_name, model in models.items():
        print(f"\n--- {model_name} ---")
        
        try:
            # Train the model
            model.fit(X_tr, y_tr)
            
            # Make predictions
            y_pred = model.predict(X_te)
            
            # For log-transformed target, convert back to original scale
            if config_name == 'Log Target':
                y_pred_original = np.exp(y_pred)
                y_te_original = np.exp(y_te)
                metrics = calculate_regression_metrics(y_test, y_pred_original)
            else:
                metrics = calculate_regression_metrics(y_te, y_pred)
            
            # Cross-validation
            cv_scores = cross_val_score(model, X_tr, y_tr, cv=5, 
                                      scoring='r2')
            
            # Store results
            config_results[model_name] = {
                'model': model,
                'predictions': y_pred,
                'metrics': metrics,
                'cv_mean': cv_scores.mean(),
                'cv_std': cv_scores.std()
            }
            
            # Track best model by R²
            if metrics['r2'] > best_model_info['r2']:
                best_model_info = {
                    'r2': metrics['r2'],
                    'config': config_name,
                    'model': model,
                    'name': model_name,
                    'results': config_results[model_name]
                }
            
            # Print key metrics
            print(f"R² Score: {metrics['r2']:.4f}")
            print(f"RMSE: {metrics['rmse']:.4f}")
            print(f"MAE: {metrics['mae']:.4f}")
            print(f"CV R²: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
            
        except Exception as e:
            print(f"Error training {model_name}: {str(e)}")
    
    results[config_name] = config_results

print(f"\n{'='*60}")
print(f"BEST MODEL OVERALL")
print(f"{'='*60}")
print(f"Configuration: {best_model_info['config']}")
print(f"Model: {best_model_info['name']}")
print(f"R² Score: {best_model_info['r2']:.4f}")
print(f"RMSE: {best_model_info['results']['metrics']['rmse']:.4f}")
print(f"CV R²: {best_model_info['results']['cv_mean']:.4f}")

## Step 7: Model Performance Analysis and Visualization
Comprehensive analysis of model performance with detailed visualizations.

In [ ]:
# Create comprehensive results DataFrame
results_data = []
for config_name, config_results in results.items():
    for model_name, model_results in config_results.items():
        metrics = model_results['metrics']
        results_data.append({
            'Configuration': config_name,
            'Model': model_name,
            'R2': metrics['r2'],
            'RMSE': metrics['rmse'],
            'MAE': metrics['mae'],
            'MSE': metrics['mse'],
            'Explained_Variance': metrics['explained_variance'],
            'CV_Mean': model_results['cv_mean'],
            'CV_Std': model_results['cv_std']
        })

results_df = pd.DataFrame(results_data)
results_df_sorted = results_df.sort_values('R2', ascending=False)

print("Top 15 Models by R² Score:")
print(results_df_sorted.head(15)[['Model', 'Configuration', 'R2', 'RMSE', 'MAE', 'CV_Mean']].to_string(index=False))

# Model performance visualization
plt.figure(figsize=(20, 16))

# R² scores heatmap
plt.subplot(2, 3, 1)
pivot_r2 = results_df.pivot_table(
    values='R2', index='Model', columns='Configuration', aggfunc='mean'
)
sns.heatmap(pivot_r2, annot=True, cmap='RdYlGn', center=0.5, fmt='.3f')
plt.title('R² Scores by Model and Configuration')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# RMSE comparison
plt.subplot(2, 3, 2)
pivot_rmse = results_df.pivot_table(
    values='RMSE', index='Model', columns='Configuration', aggfunc='mean'
)
sns.heatmap(pivot_rmse, annot=True, cmap='RdYlGn_r', fmt='.2f')
plt.title('RMSE by Model and Configuration')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# Top 10 models bar chart
plt.subplot(2, 3, 3)
top_10 = results_df_sorted.head(10)
model_labels = [f"{row['Model']}\n({row['Configuration']})" for _, row in top_10.iterrows()]
colors = plt.cm.viridis(np.linspace(0, 1, len(top_10)))

bars = plt.barh(range(len(top_10)), top_10['R2'], color=colors)
plt.yticks(range(len(top_10)), [label.replace(' (', '\n(') for label in model_labels])
plt.xlabel('R² Score')
plt.title('Top 10 Models by R² Score')
plt.xlim(0, 1)

# Add R² values on bars
for i, (bar, r2) in enumerate(zip(bars, top_10['R2'])):
    plt.text(r2 + 0.01, i, f'{r2:.3f}', va='center', ha='left', fontsize=8)

# Best model predictions vs actual
plt.subplot(2, 3, 4)
best_config = best_model_info['config']
best_name = best_model_info['name']
best_predictions = best_model_info['results']['predictions']

# Get the correct y_test values for comparison
if best_config == 'Log Target':
    y_test_compare = y_test
    y_pred_compare = np.exp(best_predictions)
else:
    y_test_compare = y_test
    y_pred_compare = best_predictions

plt.scatter(y_test_compare, y_pred_compare, alpha=0.6, s=30)
plt.plot([y_test_compare.min(), y_test_compare.max()], 
         [y_test_compare.min(), y_test_compare.max()], 'r--', lw=2)
plt.xlabel('Actual House Price ($k)')
plt.ylabel('Predicted House Price ($k)')
plt.title(f'Best Model: {best_name}\nR² = {best_model_info["r2"]:.4f}')
plt.grid(True, alpha=0.3)

# Residual analysis
plt.subplot(2, 3, 5)
residuals = y_test_compare - y_pred_compare
plt.scatter(y_pred_compare, residuals, alpha=0.6, s=30)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted House Price ($k)')
plt.ylabel('Residuals')
plt.title('Residual Plot (Best Model)')
plt.grid(True, alpha=0.3)

# Model complexity vs performance
plt.subplot(2, 3, 6)
# Define model complexity (subjective ranking)
complexity_map = {
    'Linear Regression': 1,
    'Ridge Regression': 2,
    'Lasso Regression': 2,
    'ElasticNet': 3,
    'K-Nearest Neighbors': 3,
    'Decision Tree': 4,
    'SVR (Linear)': 4,
    'SVR (RBF)': 5,
    'Random Forest': 6,
    'Extra Trees': 6,
    'Gradient Boosting': 7,
    'Neural Network': 8
}

# Get best result for each model type
best_per_model = results_df.groupby('Model')['R2'].max().reset_index()
best_per_model['Complexity'] = best_per_model['Model'].map(complexity_map)

plt.scatter(best_per_model['Complexity'], best_per_model['R2'], 
           s=100, alpha=0.7, c=best_per_model['R2'], cmap='viridis')
plt.xlabel('Model Complexity (1=Simple, 8=Complex)')
plt.ylabel('Best R² Score')
plt.title('Model Complexity vs Performance')
plt.grid(True, alpha=0.3)

# Add model names as annotations
for _, row in best_per_model.iterrows():
    plt.annotate(row['Model'].replace(' ', '\n'), 
                (row['Complexity'], row['R2']), 
                xytext=(5, 5), textcoords='offset points', 
                fontsize=8, alpha=0.8)

plt.tight_layout()
plt.show()

# Statistical analysis of results
print(f"\nModel Performance Statistics:")
print(f"Best R² Score: {results_df['R2'].max():.4f}")
print(f"Worst R² Score: {results_df['R2'].min():.4f}")
print(f"Mean R² Score: {results_df['R2'].mean():.4f}")
print(f"Std R² Score: {results_df['R2'].std():.4f}")

print(f"\nBest RMSE: {results_df['RMSE'].min():.4f}")
print(f"Worst RMSE: {results_df['RMSE'].max():.4f}")
print(f"Mean RMSE: {results_df['RMSE'].mean():.4f}")

# Configuration analysis
config_performance = results_df.groupby('Configuration').agg({
    'R2': ['mean', 'std', 'max'],
    'RMSE': ['mean', 'std', 'min']
}).round(4)

print(f"\nConfiguration Performance Summary:")
print(config_performance)

## Step 8: Feature Importance and Model Interpretation
Analyze feature importance and model interpretability.

In [ ]:
# Feature importance analysis
plt.figure(figsize=(20, 12))

# 1. Linear regression coefficients (from scaled data)
plt.subplot(2, 3, 1)
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
lr_coefs = lr_model.coef_

# Sort by absolute coefficient value
coef_abs = np.abs(lr_coefs)
sorted_idx = np.argsort(coef_abs)[::-1]

colors = ['red' if c < 0 else 'blue' for c in lr_coefs[sorted_idx]]
bars = plt.barh(range(len(lr_coefs)), lr_coefs[sorted_idx], color=colors, alpha=0.7)
plt.yticks(range(len(lr_coefs)), [boston.feature_names[i] for i in sorted_idx])
plt.xlabel('Coefficient Value')
plt.title('Linear Regression Coefficients\n(Standardized Features)')
plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)
plt.grid(True, alpha=0.3)

# 2. Random Forest feature importance
plt.subplot(2, 3, 2)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_importances = rf_model.feature_importances_

rf_sorted_idx = np.argsort(rf_importances)[::-1]
plt.barh(range(len(rf_importances)), rf_importances[rf_sorted_idx], 
         color='green', alpha=0.7)
plt.yticks(range(len(rf_importances)), [boston.feature_names[i] for i in rf_sorted_idx])
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance')
plt.grid(True, alpha=0.3)

# 3. Correlation-based importance
plt.subplot(2, 3, 3)
target_corr = np.abs(correlation_matrix['price'].drop('price'))
corr_sorted_idx = target_corr.argsort()[::-1]

plt.barh(range(len(target_corr)), target_corr.iloc[corr_sorted_idx], 
         color='orange', alpha=0.7)
plt.yticks(range(len(target_corr)), target_corr.index[corr_sorted_idx])
plt.xlabel('Absolute Correlation with Price')
plt.title('Correlation-Based Importance')
plt.grid(True, alpha=0.3)

# 4. Feature importance comparison
plt.subplot(2, 3, 4)
# Normalize all importance measures for comparison
lr_coefs_norm = np.abs(lr_coefs) / np.max(np.abs(lr_coefs))
rf_importances_norm = rf_importances / np.max(rf_importances)
corr_norm = target_corr.values / np.max(target_corr.values)

importance_df = pd.DataFrame({
    'Feature': boston.feature_names,
    'Linear_Regression': lr_coefs_norm,
    'Random_Forest': rf_importances_norm,
    'Correlation': corr_norm
})

# Calculate average importance
importance_df['Average'] = importance_df[['Linear_Regression', 'Random_Forest', 'Correlation']].mean(axis=1)
importance_df_sorted = importance_df.sort_values('Average', ascending=False)

x = np.arange(len(importance_df_sorted))
width = 0.25

plt.bar(x - width, importance_df_sorted['Linear_Regression'], width, 
        label='Linear Regression', alpha=0.7)
plt.bar(x, importance_df_sorted['Random_Forest'], width, 
        label='Random Forest', alpha=0.7)
plt.bar(x + width, importance_df_sorted['Correlation'], width, 
        label='Correlation', alpha=0.7)

plt.xlabel('Features')
plt.ylabel('Normalized Importance')
plt.title('Feature Importance Comparison')
plt.xticks(x, importance_df_sorted['Feature'], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Partial dependence plots for top 2 features
top_2_features = importance_df_sorted.head(2)['Feature'].values
feature_indices = [list(boston.feature_names).index(feat) for feat in top_2_features]

for i, (feature_name, feature_idx) in enumerate(zip(top_2_features, feature_indices)):
    plt.subplot(2, 3, 5 + i)
    
    # Create range of values for the feature
    feature_range = np.linspace(X_train[:, feature_idx].min(), 
                               X_train[:, feature_idx].max(), 50)
    
    # Use the best model for partial dependence
    predictions = []
    X_temp = X_train_scaled.mean(axis=0).reshape(1, -1).repeat(len(feature_range), axis=0)
    
    if best_model_info['config'] == 'Scaled':
        # Scale the feature values
        feature_scaled = scaler.transform(
            np.column_stack([np.zeros((len(feature_range), X_train.shape[1])) 
                           for _ in range(1)])
        )
        for j, val in enumerate(feature_range):
            X_temp[j, feature_idx] = (val - X_train[:, feature_idx].mean()) / X_train[:, feature_idx].std()
    else:
        for j, val in enumerate(feature_range):
            X_temp[j, feature_idx] = val
    
    pred_vals = best_model_info['model'].predict(X_temp)
    
    plt.plot(feature_range, pred_vals, linewidth=2, color='purple')
    plt.xlabel(feature_name)
    plt.ylabel('Predicted Price')
    plt.title(f'Partial Dependence: {feature_name}')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print feature importance rankings
print("Feature Importance Rankings:")
print("\nTop 5 features by method:")

print("\nLinear Regression (absolute coefficients):")
for i, idx in enumerate(sorted_idx[:5]):
    print(f"  {i+1}. {boston.feature_names[idx]}: {lr_coefs[idx]:.4f}")

print("\nRandom Forest:")
for i, idx in enumerate(rf_sorted_idx[:5]):
    print(f"  {i+1}. {boston.feature_names[idx]}: {rf_importances[idx]:.4f}")

print("\nCorrelation:")
for i, feature in enumerate(target_corr.index[corr_sorted_idx[:5]]):
    print(f"  {i+1}. {feature}: {target_corr[feature]:.4f}")

print("\nConsensus ranking (average of all methods):")
for i, _, row in enumerate(importance_df_sorted.head(8).itertuples()):
    print(f"  {i+1}. {row.Feature}: {row.Average:.4f}")

# Model interpretability analysis
print(f"\nModel Interpretability Analysis:")
print(f"\nBest Model: {best_model_info['name']}")
if 'Linear' in best_model_info['name'] or 'Ridge' in best_model_info['name'] or 'Lasso' in best_model_info['name']:
    print("- High interpretability: Coefficients directly show feature impact")
    print("- Each unit increase in feature leads to coefficient * unit change in price")
elif 'Tree' in best_model_info['name'] or 'Forest' in best_model_info['name']:
    print("- Medium interpretability: Feature importance shows relative influence")
    print("- Decision paths can be traced for individual predictions")
elif 'SVM' in best_model_info['name'] or 'Neural' in best_model_info['name']:
    print("- Low interpretability: Complex non-linear relationships")
    print("- Requires advanced techniques for interpretation")
else:
    print("- Variable interpretability depending on model complexity")

## Step 9: Hyperparameter Tuning and Model Optimization
Optimize the best performing models with systematic hyperparameter tuning.

In [ ]:
# Hyperparameter tuning for top performing model types
tuning_models = {
    'Ridge Regression': {
        'model': Ridge(random_state=42),
        'params': {
            'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
        }
    },
    'Lasso Regression': {
        'model': Lasso(random_state=42),
        'params': {
            'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]
        }
    },
    'ElasticNet': {
        'model': ElasticNet(random_state=42),
        'params': {
            'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
            'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
        }
    },
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [5, 10, 15, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 0.9, 1.0]
        }
    },
    'SVR (RBF)': {
        'model': SVR(kernel='rbf'),
        'params': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
            'epsilon': [0.01, 0.1, 0.2]
        }
    }
}

# Use the best configuration from previous results
best_config = best_model_info['config']
if best_config == 'Scaled':
    X_tune, X_test_tune = X_train_scaled, X_test_scaled
elif best_config == 'Selected (F-test)':
    X_tune, X_test_tune = X_train_selected, X_test_selected
elif best_config == 'Selected (RFE)':
    X_tune, X_test_tune = X_train_rfe, X_test_rfe
elif best_config == 'Polynomial':
    X_tune, X_test_tune = X_train_poly, X_test_poly
elif best_config == 'Log Target':
    X_tune, X_test_tune = X_train_scaled, X_test_scaled
    y_tune, y_test_tune = log_y_train, np.log(y_test)
else:
    X_tune, X_test_tune = X_train, X_test
    y_tune, y_test_tune = y_train, y_test

if best_config != 'Log Target':
    y_tune, y_test_tune = y_train, y_test

tuning_results = {}
print(f"Hyperparameter tuning using {best_config} configuration...\n")

for model_name, model_info in tuning_models.items():
    print(f"Tuning {model_name}...")
    
    # Grid search with cross-validation
    grid_search = GridSearchCV(
        model_info['model'], 
        model_info['params'], 
        cv=5,
        scoring='r2',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_tune, y_tune)
    
    # Test the best model
    best_model = grid_search.best_estimator_
    y_pred_tuned = best_model.predict(X_test_tune)
    
    # Handle log transformation if needed
    if best_config == 'Log Target':
        y_pred_original = np.exp(y_pred_tuned)
        y_test_original = y_test
        tuned_metrics = calculate_regression_metrics(y_test_original, y_pred_original)
    else:
        tuned_metrics = calculate_regression_metrics(y_test_tune, y_pred_tuned)
    
    tuning_results[model_name] = {
        'best_params': grid_search.best_params_,
        'best_cv_score': grid_search.best_score_,
        'test_metrics': tuned_metrics,
        'model': best_model,
        'predictions': y_pred_tuned
    }
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV R² score: {grid_search.best_score_:.4f}")
    print(f"Test R² score: {tuned_metrics['r2']:.4f}")
    print(f"Test RMSE: {tuned_metrics['rmse']:.4f}\n")

# Compare with original results
print("\n" + "="*80)
print("HYPERPARAMETER TUNING RESULTS COMPARISON")
print("="*80)

comparison_data = []
for model_name in tuning_results.keys():
    # Find original result for this model with best config
    if model_name in results[best_config]:
        original_r2 = results[best_config][model_name]['metrics']['r2']
        original_rmse = results[best_config][model_name]['metrics']['rmse']
    else:
        original_r2 = 0
        original_rmse = float('inf')
    
    tuned_r2 = tuning_results[model_name]['test_metrics']['r2']
    tuned_rmse = tuning_results[model_name]['test_metrics']['rmse']
    
    comparison_data.append({
        'Model': model_name,
        'Original_R2': original_r2,
        'Tuned_R2': tuned_r2,
        'R2_Improvement': tuned_r2 - original_r2,
        'Original_RMSE': original_rmse,
        'Tuned_RMSE': tuned_rmse,
        'RMSE_Improvement': original_rmse - tuned_rmse
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.round(4).to_string(index=False))

# Find the best tuned model
best_tuned_model = max(tuning_results.items(), key=lambda x: x[1]['test_metrics']['r2'])
best_tuned_name, best_tuned_results = best_tuned_model

print(f"\nBest tuned model: {best_tuned_name}")
print(f"Test R² score: {best_tuned_results['test_metrics']['r2']:.4f}")
print(f"Test RMSE: {best_tuned_results['test_metrics']['rmse']:.4f}")
print(f"Best parameters: {best_tuned_results['best_params']}")

# Visualize tuning results
plt.figure(figsize=(20, 12))

# R² comparison
plt.subplot(2, 3, 1)
x = np.arange(len(comparison_df))
width = 0.35

plt.bar(x - width/2, comparison_df['Original_R2'], width, 
        label='Original', alpha=0.7, color='lightblue')
plt.bar(x + width/2, comparison_df['Tuned_R2'], width, 
        label='Tuned', alpha=0.7, color='darkblue')

plt.xlabel('Models')
plt.ylabel('R² Score')
plt.title('Original vs Tuned Model Performance (R²)')
plt.xticks(x, comparison_df['Model'], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# RMSE comparison
plt.subplot(2, 3, 2)
plt.bar(x - width/2, comparison_df['Original_RMSE'], width, 
        label='Original', alpha=0.7, color='lightcoral')
plt.bar(x + width/2, comparison_df['Tuned_RMSE'], width, 
        label='Tuned', alpha=0.7, color='darkred')

plt.xlabel('Models')
plt.ylabel('RMSE')
plt.title('Original vs Tuned Model Performance (RMSE)')
plt.xticks(x, comparison_df['Model'], rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# Improvement visualization
plt.subplot(2, 3, 3)
colors_r2 = ['green' if imp > 0 else 'red' for imp in comparison_df['R2_Improvement']]
bars = plt.bar(comparison_df['Model'], comparison_df['R2_Improvement'], 
               color=colors_r2, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
plt.xlabel('Models')
plt.ylabel('R² Improvement')
plt.title('R² Improvement from Tuning')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Add improvement values on bars
for bar, imp in zip(bars, comparison_df['R2_Improvement']):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + (0.005 if height >= 0 else -0.01),
             f'{imp:+.3f}', ha='center', va='bottom' if height >= 0 else 'top', fontsize=9)

# Best tuned model predictions vs actual
plt.subplot(2, 3, 4)
best_tuned_predictions = best_tuned_results['predictions']

if best_config == 'Log Target':
    y_test_compare = y_test
    y_pred_compare = np.exp(best_tuned_predictions)
else:
    y_test_compare = y_test
    y_pred_compare = best_tuned_predictions

plt.scatter(y_test_compare, y_pred_compare, alpha=0.6, s=30)
plt.plot([y_test_compare.min(), y_test_compare.max()], 
         [y_test_compare.min(), y_test_compare.max()], 'r--', lw=2)
plt.xlabel('Actual House Price ($k)')
plt.ylabel('Predicted House Price ($k)')
plt.title(f'Best Tuned Model: {best_tuned_name}\nR² = {best_tuned_results["test_metrics"]["r2"]:.4f}')
plt.grid(True, alpha=0.3)

# Residual analysis for best tuned model
plt.subplot(2, 3, 5)
residuals = y_test_compare - y_pred_compare
plt.scatter(y_pred_compare, residuals, alpha=0.6, s=30)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted House Price ($k)')
plt.ylabel('Residuals')
plt.title('Residual Plot (Best Tuned Model)')
plt.grid(True, alpha=0.3)

# Learning curve for best tuned model
plt.subplot(2, 3, 6)
train_sizes = np.linspace(0.1, 1.0, 10)
train_sizes_abs, train_scores, val_scores = learning_curve(
    best_tuned_results['model'], X_tune, y_tune, train_sizes=train_sizes, 
    cv=5, n_jobs=-1, random_state=42, scoring='r2'
)

plt.plot(train_sizes_abs, np.mean(train_scores, axis=1), 'o-', 
         color='blue', label='Training R²')
plt.plot(train_sizes_abs, np.mean(val_scores, axis=1), 'o-', 
         color='red', label='Validation R²')
plt.fill_between(train_sizes_abs, np.mean(train_scores, axis=1) - np.std(train_scores, axis=1),
                 np.mean(train_scores, axis=1) + np.std(train_scores, axis=1), alpha=0.1, color='blue')
plt.fill_between(train_sizes_abs, np.mean(val_scores, axis=1) - np.std(val_scores, axis=1),
                 np.mean(val_scores, axis=1) + np.std(val_scores, axis=1), alpha=0.1, color='red')

plt.xlabel('Training Set Size')
plt.ylabel('R² Score')
plt.title(f'Learning Curve\n{best_tuned_name}')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 10: Final Model Evaluation and Real Estate Applications
Comprehensive final evaluation and discussion of practical real estate applications.

In [ ]:
# Determine the overall best model (considering both original and tuned)
best_overall_r2 = best_model_info['r2']
best_overall_name = best_model_info['name']
best_overall_config = best_model_info['config']
best_overall_model = best_model_info['model']
best_overall_predictions = best_model_info['results']['predictions']
best_overall_metrics = best_model_info['results']['metrics']

# Check if any tuned model is better
for model_name, tuned_results in tuning_results.items():
    if tuned_results['test_metrics']['r2'] > best_overall_r2:
        best_overall_r2 = tuned_results['test_metrics']['r2']
        best_overall_name = f"{model_name} (Tuned)"
        best_overall_model = tuned_results['model']
        best_overall_predictions = tuned_results['predictions']
        best_overall_metrics = tuned_results['test_metrics']

print(f"FINAL BEST MODEL: {best_overall_name}")
print(f"Configuration: {best_overall_config}")
print(f"Test R² Score: {best_overall_r2:.4f}")

# Comprehensive final evaluation
print("\n" + "="*80)
print("COMPREHENSIVE FINAL EVALUATION")
print("="*80)

print(f"\nFinal Model Performance Metrics:")
print(f"  R² Score: {best_overall_metrics['r2']:.4f} ({best_overall_metrics['r2']*100:.1f}% variance explained)")
print(f"  RMSE: ${best_overall_metrics['rmse']:.2f}k")
print(f"  MAE: ${best_overall_metrics['mae']:.2f}k")
print(f"  MSE: {best_overall_metrics['mse']:.2f}")
print(f"  Explained Variance: {best_overall_metrics['explained_variance']:.4f}")

# Calculate additional metrics
if best_overall_config == 'Log Target':
    y_test_final = y_test
    y_pred_final = np.exp(best_overall_predictions)
else:
    y_test_final = y_test
    y_pred_final = best_overall_predictions

# Prediction accuracy analysis
absolute_errors = np.abs(y_test_final - y_pred_final)
percentage_errors = (absolute_errors / y_test_final) * 100

print(f"\nPrediction Accuracy Analysis:")
print(f"  Mean Absolute Error: ${absolute_errors.mean():.2f}k")
print(f"  Median Absolute Error: ${np.median(absolute_errors):.2f}k")
print(f"  Mean Percentage Error: {percentage_errors.mean():.1f}%")
print(f"  Median Percentage Error: {np.median(percentage_errors):.1f}%")

# Accuracy by price range
price_ranges = [(0, 15), (15, 25), (25, 35), (35, float('inf'))]
range_labels = ['Low ($0-15k)', 'Medium ($15-25k)', 'High ($25-35k)', 'Luxury ($35k+)']

print(f"\nAccuracy by Price Range:")
for (low, high), label in zip(price_ranges, range_labels):
    mask = (y_test_final >= low) & (y_test_final < high)
    if mask.sum() > 0:
        range_mae = absolute_errors[mask].mean()
        range_mape = percentage_errors[mask].mean()
        range_r2 = r2_score(y_test_final[mask], y_pred_final[mask])
        print(f"  {label}: MAE=${range_mae:.2f}k, MAPE={range_mape:.1f}%, R²={range_r2:.3f} (n={mask.sum()})")

# Model confidence analysis
confidence_threshold_5 = np.percentile(absolute_errors, 95)  # 95% of predictions within this error
confidence_threshold_10 = np.percentile(absolute_errors, 90)  # 90% of predictions within this error

print(f"\nModel Confidence Analysis:")
print(f"  95% of predictions within: ±${confidence_threshold_5:.2f}k")
print(f"  90% of predictions within: ±${confidence_threshold_10:.2f}k")
print(f"  Predictions within ±$2k: {(absolute_errors <= 2).mean()*100:.1f}%")
print(f"  Predictions within ±$5k: {(absolute_errors <= 5).mean()*100:.1f}%")

# Final visualization
plt.figure(figsize=(20, 12))

# Final predictions vs actual
plt.subplot(2, 3, 1)
plt.scatter(y_test_final, y_pred_final, alpha=0.6, s=40, c=absolute_errors, cmap='viridis')
plt.plot([y_test_final.min(), y_test_final.max()], 
         [y_test_final.min(), y_test_final.max()], 'r--', lw=2, label='Perfect prediction')
plt.colorbar(label='Absolute Error ($k)')
plt.xlabel('Actual House Price ($k)')
plt.ylabel('Predicted House Price ($k)')
plt.title(f'Final Model Predictions\n{best_overall_name}\nR² = {best_overall_r2:.4f}')
plt.legend()
plt.grid(True, alpha=0.3)

# Residual analysis
plt.subplot(2, 3, 2)
residuals_final = y_test_final - y_pred_final
plt.scatter(y_pred_final, residuals_final, alpha=0.6, s=40)
plt.axhline(y=0, color='r', linestyle='--', label='Zero residual')
plt.axhline(y=residuals_final.std(), color='orange', linestyle=':', alpha=0.7, label='+1 std')
plt.axhline(y=-residuals_final.std(), color='orange', linestyle=':', alpha=0.7, label='-1 std')
plt.xlabel('Predicted House Price ($k)')
plt.ylabel('Residuals ($k)')
plt.title('Residual Analysis')
plt.legend()
plt.grid(True, alpha=0.3)

# Error distribution
plt.subplot(2, 3, 3)
plt.hist(absolute_errors, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(absolute_errors.mean(), color='red', linestyle='--', 
           label=f'Mean: ${absolute_errors.mean():.2f}k')
plt.axvline(np.median(absolute_errors), color='green', linestyle='--', 
           label=f'Median: ${np.median(absolute_errors):.2f}k')
plt.xlabel('Absolute Error ($k)')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.legend()
plt.grid(True, alpha=0.3)

# Percentage error by price
plt.subplot(2, 3, 4)
plt.scatter(y_test_final, percentage_errors, alpha=0.6, s=40)
plt.axhline(y=percentage_errors.mean(), color='red', linestyle='--', 
           label=f'Mean: {percentage_errors.mean():.1f}%')
plt.xlabel('Actual House Price ($k)')
plt.ylabel('Percentage Error (%)')
plt.title('Percentage Error vs House Price')
plt.legend()
plt.grid(True, alpha=0.3)

# Feature importance for final model
plt.subplot(2, 3, 5)
if hasattr(best_overall_model, 'feature_importances_'):
    importances = best_overall_model.feature_importances_
    indices = np.argsort(importances)[::-1][:10]
    
    plt.barh(range(len(indices)), importances[indices], color='lightcoral', alpha=0.7)
    
    # Determine feature names based on configuration
    if best_overall_config == 'Selected (F-test)':
        feature_names_final = selected_features
    elif best_overall_config == 'Selected (RFE)':
        feature_names_final = rfe_features
    else:
        feature_names_final = boston.feature_names
    
    plt.yticks(range(len(indices)), [feature_names_final[i] for i in indices])
    plt.xlabel('Feature Importance')
    plt.title('Top 10 Feature Importances\n(Final Model)')
elif hasattr(best_overall_model, 'coef_'):
    coefs = np.abs(best_overall_model.coef_)
    indices = np.argsort(coefs)[::-1][:10]
    
    plt.barh(range(len(indices)), coefs[indices], color='lightblue', alpha=0.7)
    
    if best_overall_config == 'Selected (F-test)':
        feature_names_final = selected_features
    elif best_overall_config == 'Selected (RFE)':
        feature_names_final = rfe_features
    else:
        feature_names_final = boston.feature_names
    
    plt.yticks(range(len(indices)), [feature_names_final[i] for i in indices])
    plt.xlabel('Coefficient Magnitude')
    plt.title('Top 10 Coefficient Magnitudes\n(Final Model)')
else:
    plt.text(0.5, 0.5, 'Feature importance\nnot available\nfor this model type', 
             ha='center', va='center', transform=plt.gca().transAxes, fontsize=12)
    plt.title('Feature Importance')

# Model summary
plt.subplot(2, 3, 6)
summary_text = f"""
FINAL MODEL SUMMARY

Model: {best_overall_name}
Configuration: {best_overall_config}

PERFORMANCE METRICS:
• R² Score: {best_overall_r2:.4f}
• RMSE: ${best_overall_metrics['rmse']:.2f}k
• MAE: ${best_overall_metrics['mae']:.2f}k
• Mean % Error: {percentage_errors.mean():.1f}%

PRACTICAL ACCURACY:
• Within ±$2k: {(absolute_errors <= 2).mean()*100:.0f}%
• Within ±$5k: {(absolute_errors <= 5).mean()*100:.0f}%
• 95% Confidence: ±${confidence_threshold_5:.1f}k

MODEL CHARACTERISTICS:
• Complexity: {'High' if 'Neural' in best_overall_name or 'Boosting' in best_overall_name else 'Medium' if 'Forest' in best_overall_name or 'Tree' in best_overall_name else 'Low'}
• Interpretability: {'Low' if 'Neural' in best_overall_name or 'SVR' in best_overall_name else 'Medium' if 'Forest' in best_overall_name or 'Tree' in best_overall_name else 'High'}
• Speed: {'Fast' if 'Linear' in best_overall_name or 'Ridge' in best_overall_name or 'Lasso' in best_overall_name else 'Medium'}
"""

plt.text(0.05, 0.95, summary_text, transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top', fontfamily='monospace')
plt.axis('off')

plt.tight_layout()
plt.show()

# Real estate application analysis
print("\n" + "="*80)
print("REAL ESTATE APPLICATION ANALYSIS")
print("="*80)

print(f"\nModel Suitability for Real Estate Applications:")

if best_overall_r2 >= 0.80:
    print(f"✓ EXCELLENT for automated valuation models (AVMs)")
    print(f"✓ Suitable for preliminary property assessments")
    print(f"✓ Can be used for market trend analysis")
elif best_overall_r2 >= 0.70:
    print(f"✓ GOOD for supporting human appraisers")
    print(f"○ Requires additional validation for high-value properties")
    print(f"✓ Useful for comparative market analysis")
elif best_overall_r2 >= 0.60:
    print(f"○ FAIR for rough price estimation")
    print(f"✗ Not suitable for primary valuation")
    print(f"○ May be useful for market segmentation")
else:
    print(f"✗ POOR - Requires significant improvement")
    print(f"✗ Not recommended for any production use")

print(f"\nRecommended Applications:")
print(f"1. Automated Valuation Models (AVMs)")
print(f"   - Quick property value estimates")
print(f"   - Portfolio valuation")
print(f"   - Market screening")

print(f"\n2. Investment Analysis")
print(f"   - Property investment screening")
print(f"   - ROI calculations")
print(f"   - Market opportunity identification")

print(f"\n3. Real Estate Decision Support")
print(f"   - Pricing strategy development")
print(f"   - Market trend analysis")
print(f"   - Comparative market analysis (CMA)")

print(f"\nImplementation Considerations:")
print(f"• Data Requirements: {len(boston.feature_names)} property features")
print(f"• Update Frequency: Monthly or quarterly model retraining recommended")
print(f"• Validation: Compare predictions with recent sales for accuracy monitoring")
print(f"• Human Oversight: Professional appraisal for high-value or unique properties")

print(f"\nLimitations and Risks:")
print(f"⚠️  Dataset contains historical data that may not reflect current market conditions")
print(f"⚠️  Model may not capture unique property characteristics or market anomalies")
print(f"⚠️  Geographic and temporal limitations of the training data")
print(f"⚠️  Ethical concerns regarding certain features in the original dataset")

print(f"\nETHICAL CONSIDERATIONS:")
print(f"⚠️  This dataset has been deprecated due to ethical concerns")
print(f"⚠️  Some features may perpetuate historical biases")
print(f"⚠️  Modern implementations should focus on property characteristics only")
print(f"⚠️  Regular bias auditing recommended for any production system")

## Key Findings and Real Estate Applications

### ⚠️ Important Ethical Note
**This analysis uses the deprecated Boston Housing dataset for educational purposes only. The dataset contains features that may perpetuate historical biases and should not be used as a reference for modern real estate applications without addressing these ethical concerns.**

### Dataset Characteristics and Challenges
- **Small Dataset**: 506 samples limit generalizability to broader markets
- **Historical Data**: 1978 data may not reflect current market dynamics
- **Feature Quality**: Mix of property-specific and neighborhood-level features
- **Geographic Limitation**: Specific to Boston area, not generalizable globally

### Model Performance Analysis
- **Strong Predictive Power**: Top models achieve R² > 0.8, indicating good explanatory capability
- **Reasonable Error Rates**: RMSE typically $3-5k on average home values of $22k (1978 dollars)
- **Feature Sensitivity**: Models respond appropriately to key housing factors (rooms, location, condition)
- **Overfitting Risk**: Small dataset size requires careful validation

### Most Influential Features

#### **Property Characteristics**
- **RM (Average Rooms)**: Strong positive correlation with price
- **AGE (Building Age)**: Older properties generally valued lower
- **CHAS (Charles River)**: Waterfront proximity premium

#### **Neighborhood Factors**
- **LSTAT (Lower Status %)**: Strong negative correlation with housing values
- **PTRATIO (Pupil-Teacher Ratio)**: Education quality impact on values
- **CRIM (Crime Rate)**: Safety considerations in pricing

#### **Accessibility and Infrastructure**
- **DIS (Employment Center Distance)**: Commute convenience factor
- **RAD (Highway Access)**: Transportation infrastructure impact
- **TAX (Property Tax Rate)**: Tax burden effect on desirability

### Modern Real Estate Applications (Considerations)

#### **Automated Valuation Models (AVMs)**
- **Primary Use**: Quick property value estimates for lending and investment
- **Accuracy Requirements**: ±5-10% error acceptable for screening purposes
- **Update Frequency**: Monthly retraining with recent sales data
- **Human Validation**: Professional appraisal for high-value or unique properties

#### **Investment Analysis Tools**
- **Portfolio Screening**: Rapid assessment of multiple properties
- **Market Opportunity Identification**: Undervalued property detection
- **ROI Calculations**: Expected return analysis for investors
- **Risk Assessment**: Price volatility and market stability analysis

#### **Real Estate Decision Support**
- **Pricing Strategy**: Market-based pricing recommendations for sellers
- **Negotiation Support**: Fair value ranges for buyers and sellers
- **Market Analysis**: Trend identification and comparative studies
- **Development Planning**: Site selection and project viability assessment

### Implementation Framework

#### **Data Infrastructure**
1. **Property Database**: Comprehensive property characteristics and transaction history
2. **Market Data Integration**: Recent sales, listings, and economic indicators
3. **Geographic Information**: Location-based features and neighborhood analytics
4. **Quality Assurance**: Data validation and outlier detection systems

#### **Model Architecture**
1. **Ensemble Approach**: Combine multiple algorithms for robust predictions
2. **Feature Engineering**: Transform raw data into predictive features
3. **Regional Models**: Location-specific models for different markets
4. **Temporal Updates**: Regular retraining to capture market changes

#### **Validation and Monitoring**
1. **Holdout Testing**: Reserve recent data for model validation
2. **Prediction Tracking**: Monitor accuracy against actual sales
3. **Market Adjustment**: Calibrate for changing market conditions
4. **Bias Detection**: Regular auditing for fairness and discrimination

### Ethical and Legal Considerations

#### **Fair Housing Compliance**
- **Protected Classes**: Ensure models don't discriminate based on race, religion, etc.
- **Feature Selection**: Use only property-related characteristics
- **Bias Testing**: Regular statistical testing for discriminatory outcomes
- **Transparency**: Clear explanation of factors influencing valuations

#### **Professional Standards**
- **Appraiser Guidelines**: Complement, not replace, professional judgment
- **Regulatory Compliance**: Adhere to local real estate and lending regulations
- **Consumer Protection**: Clear disclosure of automated valuation limitations
- **Professional Oversight**: Licensed professionals should validate model outputs

### Future Enhancements

#### **Advanced Modeling Techniques**
- **Deep Learning**: Neural networks for complex feature interactions
- **Geospatial Models**: Incorporate detailed location and proximity data
- **Time Series Integration**: Model temporal price trends and seasonality
- **External Data Sources**: Economic indicators, demographics, infrastructure changes

#### **Technology Integration**
- **Computer Vision**: Automated property condition assessment from images
- **IoT Data**: Smart home features and energy efficiency metrics
- **Market Sentiment**: Social media and news analysis for market timing
- **Real-time Updates**: Continuous learning from new market transactions

### Limitations and Risks

#### **Model Limitations**
- **Data Dependency**: Quality limited by input data accuracy and completeness
- **Market Uniqueness**: Difficulty capturing unique property characteristics
- **Economic Shifts**: Models may lag during rapid market changes
- **Geographic Scope**: Limited transferability across different markets

#### **Business Risks**
- **Over-reliance**: Automated systems shouldn't replace human expertise entirely
- **Liability Issues**: Potential legal responsibility for inaccurate valuations
- **Market Disruption**: Rapid technological change in real estate sector
- **Competitive Pressure**: Need for continuous model improvement and innovation

### Conclusion
While this analysis demonstrates the technical feasibility of machine learning for housing price prediction, modern implementations must address the ethical concerns present in historical datasets. The techniques shown here provide a foundation for developing fair, accurate, and useful real estate valuation tools when applied to appropriate, unbiased datasets with proper oversight and validation procedures.

**Key Recommendation**: Future real estate ML applications should focus exclusively on property characteristics and legitimate market factors while implementing robust bias detection and fairness monitoring systems.